<a href="https://colab.research.google.com/github/shresh15/VisionAQI/blob/main/AQI_Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle (2).json to kaggle (2).json


{'kaggle (2).json': b'{"username":"shresh15","key":"18ea0ec18989bc6b7372f83e8c47c54c"}'}

In [2]:
!ls


'kaggle (2).json'   sample_data


In [3]:
!mkdir -p /root/.config/kaggle

In [6]:
!mv "kaggle (2).json" /root/.config/kaggle/kaggle.json

In [7]:
!chmod 600 /root/.config/kaggle/kaggle.json

In [8]:
!kaggle datasets list | head

ref                                               title                                           size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------  ----------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
neurocipher/heartdisease                          Heart Disease                                   3491  2025-12-11 15:29:14.327000           2114        172  1.0              
neurocipher/student-performance                   Student Performance                            49705  2025-12-12 12:06:28.973000           1261        107  1.0              
dansbecker/powerlifting-database                  powerlifting-database                        9277600  2019-04-30 21:07:41.560000          24255        351  0.5882353        
rtatman/188-million-us-wildfires                  1.88 Million US Wildfires                  176270559  2020-05-12 21:03

In [9]:
!kaggle datasets download kmljts/reside-6k

Dataset URL: https://www.kaggle.com/datasets/kmljts/reside-6k
License(s): apache-2.0
100% 1.48G/1.48G [00:18<00:00, 379MB/s]
100% 1.48G/1.48G [00:18<00:00, 87.3MB/s]


In [10]:
!unzip reside-6k.zip -d /content/RESIDE-6K

Streaming output truncated to the last 5000 lines.
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/19.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/190.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1900.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1901.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1902.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1903.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1904.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1905.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1906.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1907.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1908.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1909.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/191.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1910.jpg  
  inflating: /content/RESIDE-6K/RESIDE-6K/train/hazy/1911

In [11]:
!ls /content/RESIDE-6K/RESIDE-6K

test  train


In [12]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [13]:
class HazeDataset(Dataset):
    def __init__(self, hazy_dir, clear_dir, transform=None):
        self.hazy_dir = hazy_dir
        self.clear_dir = clear_dir
        self.transform = transform
        self.images = os.listdir(hazy_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        hazy_path = os.path.join(self.hazy_dir, self.images[idx])
        clear_path = os.path.join(self.clear_dir, self.images[idx])

        hazy = Image.open(hazy_path).convert("RGB")
        clear = Image.open(clear_path).convert("RGB")

        if self.transform:
            hazy = self.transform(hazy)
            clear = self.transform(clear)

        return hazy, clear

In [14]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])


In [15]:
dataset = HazeDataset(
    hazy_dir="RESIDE-6K/RESIDE-6K/train/hazy",
    clear_dir="RESIDE-6K/RESIDE-6K/train/GT",
    transform=transform
)

loader = DataLoader(dataset, batch_size=8, shuffle=True)

AOD NET TRAINING

In [16]:
class AODNet(nn.Module):
    def __init__(self):
        super(AODNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 3, 3, padding=1)
        self.conv2 = nn.Conv2d(3, 3, 3, padding=1)
        self.conv3 = nn.Conv2d(3, 3, 3, padding=1)
        self.conv4 = nn.Conv2d(3, 3, 3, padding=1)
        self.conv5 = nn.Conv2d(12, 3, 3, padding=1)

        self.relu = nn.ReLU()

    def forward(self, x):
        x1 = self.relu(self.conv1(x))
        x2 = self.relu(self.conv2(x1))
        x3 = self.relu(self.conv3(x2))
        x4 = self.relu(self.conv4(x3))

        concat = torch.cat([x1, x2, x3, x4], dim=1)
        K = self.relu(self.conv5(concat))

        J = K * x - K + 1
        return J,K



In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AODNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
